In [106]:
import spotipy
import spotipy.util as util
sp = spotipy.Spotify() 
from spotipy.oauth2 import SpotifyClientCredentials

from datetime import datetime
from datetime import timedelta

import pandas as pd

## spotify auth flow

In [107]:
cid ="1c7e8aed94914da78a7b264590d7fc21" 
secret = "c2113b0c6e3543d282adb8c61e4abede"
username = "trudypaintet"
redirect_uri="http://localhost:3000"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

scope = 'user-read-recently-played'
token = util.prompt_for_user_token(username,scope,cid,secret,redirect_uri)
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

## download recently played songs
1. collect songs info
    - name, id, time played
2. filter for UNINDEXED songs from TODAY
    - script will run every hour and add new entries to a csv for the date

In [122]:
recent_songs = sp.current_user_recently_played()

count = 0
track_ids = []
mood_avgs = []

#creating data frame to add to the csv
song_dict = {'image':[],
    'song_name': [],
             'album_name':[],
        'artist_name': [],
        'track_id':[],
        'played_at':[],
        'valence':[],
        'danceability':[],
        'energy':[],
        'mood_avg':[]}

for song in recent_songs["items"]:
    
    #break out of loop if the song was not played in the last hour
    time_played = datetime.strptime(song["played_at"],"%Y-%m-%dT%H:%M:%S.%fZ")
    time_elapsed = time_played - datetime.now() 
    if time_elapsed < timedelta(hours=3, minutes=1):
        break
    
    #spotify has weird hours/time zone going on
    #and i need to make it pretty for the table
    adjusted_played_at = time_played - timedelta(hours=4)
    readable_time = adjusted_played_at.strftime("%H") + ":" + adjusted_played_at.strftime("%M")
    
    count+=1
    
    features = sp.audio_features(song["track"]["id"])
    valence = float(features[0]["valence"])
    dance = float(features[0]["danceability"])
    energy = float(features[0]["energy"])
    avg = (valence*0.4) + (dance*0.4) + (energy*0.2)
    mood_avgs.append(avg)
    
    
    #adding songs to dictionary to be put into dataframe
    song_dict['image'].append(song['track']['album']['images'][0]['url'])
    song_dict['song_name'].append(song["track"]["name"])
    song_dict['album_name'].append(song["track"]["album"]["name"])
    song_dict['artist_name'].append(song["track"]["artists"][0]["name"])
    song_dict['track_id'].append(song["track"]["id"])
    song_dict['played_at'].append(readable_time)
    song_dict['valence'].append(valence)
    song_dict['danceability'].append(dance)
    song_dict['energy'].append(energy)
    song_dict['mood_avg'].append(round(avg, 3))

    
#     print("-----------",count,"---------------")
#     #BASIC INFO
#     print("Song: \t\t", song["track"]["name"], " by ", song["track"]["artists"][0]["name"])
#     print("ID: \t\t", song["track"]["id"])
#     print("Played at: \t", song["played_at"], "\n")
    
#     #MOOD INDEXES
#     #weight ->  valence - 0.5
#     #           danceability - 0.3
#     #           energy - 0.2
#     print("Valence: \t\t", features[0]["valence"])
#     print("Danceability: \t\t", features[0]["danceability"])
#     print("Energy: \t\t", features[0]["energy"])
#     print("GRAND TOTAL: \t\t", round(avg, 3))
    
# avg_mood = sum(mood_avgs)/len(mood_avgs)
# print("\n$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
# print("Average Mood Index: \t", round(avg_mood, 3))
# print("Num Songs Played: \t", len(mood_avgs))

#add spotify data to csv
df = pd.DataFrame.from_dict(song_dict,orient='index').transpose()
filename = "daily-data/" + datetime.now().strftime('%Y-%m-%d/')[:10] + ".csv"
df.to_csv(filename, mode = 'a', header = False, index = False)

    
    

In [98]:
print(datetime.time(datetime.now()))

19:10:21.805786


In [121]:
recent_songs['items'][3]['track']['album']['images'][0]['url']

'https://i.scdn.co/image/ab67616d0000b273ea9bbdf9f1433e68c7dffbad'